In [1]:
!pip install mediapipe
import cv2
import mediapipe as mp
import numpy as np
# from google.colab.patches import cv2_imshow

mp_pose = mp.solutions.pose

# Setup the Pose function for images - independently for the images standalone processing.
pose_image = mp_pose.Pose(static_image_mode=True, min_detection_confidence=0.8,model_complexity=2)

# Setup the Pose function for videos - for video processing.
pose_video = mp_pose.Pose(static_image_mode=False, min_detection_confidence=0.7,
                          min_tracking_confidence=0.7)

# Initialize mediapipe drawing class - to draw the landmarks points.
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles

def mpPoseImg(image_pose, pose=pose_image, draw=False, display=False):
    
    annotated_image = image_pose.copy()
    
    image_in_RGB = cv2.cvtColor(image_pose, cv2.COLOR_BGR2RGB)
 
    
    results = pose.process(image_in_RGB)

    if results.pose_landmarks and draw:    
      mp_drawing.draw_landmarks(image=annotated_image,landmark_list=results.pose_landmarks,connections=mp_pose.POSE_CONNECTIONS,
                                landmark_drawing_spec=mp_drawing_styles.get_default_pose_landmarks_style())


        # mp_drawing.draw_landmarks(image=original_image, landmark_list=resultant.pose_landmarks,connections=mp_pose.POSE_CONNECTIONS,
        #                           landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255),thickness=3, circle_radius=3),
        #                           connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),thickness=2, circle_radius=2))

    if display:
            
            # plt.figure(figsize=[22,22])
            # plt.subplot(121);plt.imshow(image_pose[:,:,::-1]);plt.title("Input Image");plt.axis('off');
            # plt.subplot(122);plt.imshow(annotated_image[:,:,::-1]);plt.title("Pose detected Image");plt.axis('off');
            cv2.imshow(annotated_image)
            return results

    else:
        return annotated_image,results

In [2]:
# BODY_PARTS = {
#     0: 'nose', 1: 'left_eye_inner', 2: 'left_eye', 3: 'left_eye_outer', 4: 'right_eye_inner', 5: 'right_eye', 6: 'right_eye_outer', 7: 'left_ear', 8: 'right_ear',
#     9: 'mouth_left', 10: 'mouth_right', 11: 'left_shoulder', 12: 'right_shoulder', 13: 'left_elbow', 14: 'right_elbow', 15: 'left_wrist',16: 'right_wrist',
#     17: 'left_pinky_finger', 18: 'right_pinky_finger', 19: 'left_index_finger', 20: 'right_index_finger', 21: 'left_thumb', 22: 'right_thumb', 23: 'left_hip', 24: 'right_hip',
#     25: 'left_knee', 26: 'right_knee', 27: 'left_ankle', 28: 'right_ankle', 29: 'left_heel', 30: 'right_heel', 31: 'left_foot_index', 32: 'right_foot_index', 33: 'mid_hip', 34: 'neck'
# }

BODY_PARTS = {0: 'NOSE',
              1: 'LEFT_EYE_INNER',
              2: 'LEFT_EYE',
              3: 'LEFT_EYE_OUTER',
              4: 'RIGHT_EYE_INNER',
              5: 'RIGHT_EYE',
              6: 'RIGHT_EYE_OUTER',
              7: 'LEFT_EAR',
              8: 'RIGHT_EAR',
              9: 'MOUTH_LEFT',
              10: 'MOUTH_RIGHT',
              11: 'LEFT_SHOULDER',
              12: 'RIGHT_SHOULDER',
              13: 'LEFT_ELBOW',
              14: 'RIGHT_ELBOW',
              15: 'LEFT_WRIST',
              16: 'RIGHT_WRIST',
              17: 'LEFT_PINKY',
              18: 'RIGHT_PINKY',
              19: 'LEFT_INDEX',
              20: 'RIGHT_INDEX',
              21: 'LEFT_THUMB',
              22: 'RIGHT_THUMB',
              23: 'LEFT_HIP',
              24: 'RIGHT_HIP',
              25: 'LEFT_KNEE',
              26: 'RIGHT_KNEE',
              27: 'LEFT_ANKLE',
              28: 'RIGHT_ANKLE',
              29: 'LEFT_HEEL',
              30: 'RIGHT_HEEL',
              31: 'LEFT_FOOT_INDEX',
              32: 'RIGHT_FOOT_INDEX',
              33: 'MID_HIP',
              34: 'NECK'}

BODY_PARTS_DICT = {value: key for key, value in BODY_PARTS.items()}

# Define the groups of keypoints by body parts
keypoint_groups = {
     0:[1,2,3,4,5,6,7,8],
     1:[0,2,3,4,5,6],
     2:[0,1,3,4,5,6],
     3:[0,1,2,4,5,6],
     4:[0,1,2,3,5,6],
     5:[0,1,2,3,4,6],
     6:[0,1,2,3,4,5],
     7:[0,1,2,3,9],
     8:[0,4,5,6,10],
     9:[0,7,10],
     10:[0,8,9],
     11:[12,13,9],
     12:[11,14,10],
     13:[11,15],
     14:[12,16],
     15:[13,17,19],
     16:[14,18,20],
     17:[15,19,21],
     18:[16,20,22],
     19:[15,17,21],
     20:[16,18,22],
     21:[17,19],
     22:[18,20],
     23:[24,25],
     24:[23,26],
     25:[23,27],
     26:[24,28],
     27:[25,29,31],
     28:[26,30,32],
     29:[27,31],
     30:[28,32],
     31:[27,29],
     32:[28,30],
     33:[23,24],
     34:[11,12]
}


In [3]:
def ReplacebyAverage(index,keypoints):
  group_indices = keypoint_groups[index]
  group_keypoints = [keypoints[i] for i in group_indices]
  valid_keypoints = [k for k in group_keypoints if k is not None]
  if len(valid_keypoints) > 0:
     group_average_x = np.mean([k[0] for k in valid_keypoints if k[0] is not None])
     group_average_y = np.mean([k[1] for k in valid_keypoints if k[1] is not None])
    #  keypoints[index][0]=group_average_x
    #  keypoints[index][1]=group_average_y
  else:
    # If all keypoints in the group are missing, replace with (0,0) or another default value
    group_average_x, group_average_y = 0, 0
  return [group_average_x,group_average_y]

In [4]:
def ReplaceNonebyAverage(keypoints):
  group_indices = keypoint_groups[index]
  group_keypoints = [keypoints[i] for i in group_indices]
  
  if len(valid_keypoints) > 0:
     group_average_x = np.mean([k[0] for k in valid_keypoints if k[0] is not None])
     group_average_y = np.mean([k[1] for k in valid_keypoints if k[1] is not None])
    #  keypoints[index][0]=group_average_x
    #  keypoints[index][1]=group_average_y
  else:
    # If all keypoints in the group are missing, replace with (0,0) or another default value
    group_average_x, group_average_y = 0, 0
  return [group_average_x,group_average_y]


In [5]:
# calculate the center of the object in each frame
def CentralisedKeypoints1(keypoints, live = False):
  frame_centers = []
 
  for frame_keypoints in keypoints:
    # assume the center is the mean of all keypoints
    center_x = np.nanmean([kp[0] for kp in frame_keypoints if kp[0] is not None])
    center_y = np.nanmean([kp[1] for kp in frame_keypoints if kp[1] is not None])
    frame_centers.append((center_x, center_y))

  relative_keypoints = keypoints

  for j in range(len(relative_keypoints)):               #frame no
    for k in range(len(relative_keypoints[j])):          # each keypoint of frame
      if relative_keypoints[j][k][0] is not None:
          relative_keypoints[j][k][0] -= frame_centers[j][0]
      if relative_keypoints[j][k][1] is not None:
        relative_keypoints[j][k][1] -= frame_centers[j][1]

  frame=0
  for i in range(len(relative_keypoints)):   #each frame
    countx=0
    county=0
    for j in range(len(relative_keypoints[i])):   #34 keypoints
      if relative_keypoints[i][j][0] is None:
        countx+=1
      if relative_keypoints[i][j][1] is None:
        county+=1
    if(countx== 34 and county==34):
#       print("empty")
      return []

    for j in range(len(relative_keypoints[i])):   #34 keypoints
      if relative_keypoints[i][j][0] is None:
#         print("frame : ",i)
        # ReplacebyAverage(j,relative_keypoints[i])
        relative_keypoints[i][j][0] = -1
      if relative_keypoints[i][j][1] is None:
        # ReplacebyAverage(j,relative_keypoints[i])
        relative_keypoints[i][j][1] = -1
  return relative_keypoints

In [6]:
numOfKeypoints=34

import numpy as np
import tensorflow as tf

# calculate the center of the object in each frame

def CentralisedKeypoints2(keypoints,live = False):
  frame_centers = []
  for frame_keypoints in keypoints:
    # assume the center is the mean of all keypoints
    center_x = np.nanmean([kp[0] for kp in frame_keypoints if kp[0] is not None])
    center_y = np.nanmean([kp[1] for kp in frame_keypoints if kp[1] is not None])
    frame_centers.append((center_x, center_y))


  relative_keypoints = keypoints

  for j in range(len(relative_keypoints)):               #frame no j
    for k in range(len(relative_keypoints[j])):           # k keypoint of frame j
      if relative_keypoints[j][k][0] is not None:
          relative_keypoints[j][k][0] -= frame_centers[j][0]
      if relative_keypoints[j][k][1] is not None:
        relative_keypoints[j][k][1] -= frame_centers[j][1]

    frames_keypoints=[]
    for j in range(len(relative_keypoints)):
        flag=1
        for k in range(len(relative_keypoints[j])):
          if relative_keypoints[j][k][0] is None or relative_keypoints[j][k][1] is None:
            flag=0
            break
        if(flag==1):
          frames_keypoints.append(relative_keypoints[j])

    if(len(frames_keypoints)>0):
      kk = [[0.0,0.0] for _ in range(numOfKeypoints)]
      for x in range (16-len(frames_keypoints)):
            frames_keypoints.append(kk)
      return frames_keypoints

  if live:
        return []
  return None
    


In [7]:
numOfKeypoints = 35
def getKeyPointsforFrame(frame):
  img,results = mpPoseImg(frame)
  keypoints = [[None,None] for _ in range(numOfKeypoints)]
  if results.pose_landmarks == None:
    return keypoints
    
  landmarks = results.pose_landmarks
  for i in range(numOfKeypoints-2):
     mark = landmarks.landmark[i]
     keypoints[i][0]=mark.x
     keypoints[i][1]=mark.y
  #------------------------------------------------------------------------------------------------
  try:
    # MID HIP COORDINATES
      left_hip_index = 23
      right_hip_index = 24
      Mid_hip_x =  (keypoints[left_hip_index][0]+keypoints[right_hip_index][0])/2
      Mid_hip_y =(keypoints[left_hip_index][1]+keypoints[right_hip_index][1])/2
      # keypoints.append([Mid_hip_x,Mid_hip_y])
      keypoints[33][0]=Mid_hip_x
      keypoints[33][1]=Mid_hip_y
                          
  except:
      pass
      
  #------------------------------------------------------------------------------------------------
  try:
    # NECK COORDINATES
      left_shoulder_index = 11
      right_shoulder_index = 12
      Neck_x =  (keypoints[left_shoulder_index][0]+keypoints[right_shoulder_index][0])/2
      Neck_y =(keypoints[left_shoulder_index][1]+keypoints[right_shoulder_index][1])/2
      # keypoints.append([Neck_x,Neck_y])
      keypoints[34][0]=Neck_x
      keypoints[34][1]=Neck_y
                          
  except:
      pass

  #------------------------------------------------------------------------------------------------
     
  return keypoints



In [1]:
from keras import models
RGBmodel = models.load_model('RGBStream.h5')
Posemodel = models.load_model('PoseStream2.h5')

labels=['Adhomukhasvanasana',
  'Ardhachakrasana',
  'Bhujangasana',
  'Dhanurasana',
  'Marjariasana',
  'Padahastasana',
  'Padmasana',
  'Pawanmuktasana',
  'Phalakasana',
  'Sarvangasana',
  'Sashankasana',
  'Setubandhasana',
  'Shavasana',
  'Tadasana',
  'Trikonasana',
  'Ustrasana',
  'Vakrasana',
  'Virbhadrasana1',
  'Virbhadrasana2',
  'Vrikshasana']
  

def predictYogaPose(video,keypoints,integer = False,RGBRatio = 0.2, PoseRatio = 0.8):
  softmaxRGB = RGBmodel.predict(video)
  softmaxPose = Posemodel.predict(keypoints)
  combined_softmax = (softmaxRGB[0]*RGBRatio + softmaxPose[0]*PoseRatio) 
  
  predicted_labels = np.argmax(combined_softmax)
  
  if(integer == True):
    return predicted_labels
#   print(combined_softmax)
  # print(max(combined_softmax))
  if max(combined_softmax) < 0.75:
    return ""
  return labels[predicted_labels]